
# Fourth & Value — QC Slim: Actuals vs Model

**Why this exists:** Your `data/actuals/weekX.csv` is already in **long format** (`player_key,name_std,market_std,side,point,actual_value,result`).
So we don't need to hunt for per-stat columns; we can grade directly. This notebook is the **clean minimal path** with an
optional **stats backfill** for O/U if the actuals file doesn't include those yet.

**Outputs**
- `data/eval/grades_week{WEEK}.csv` (row-level graded)
- `data/eval/grades_week{WEEK}_by_market.csv`
- `data/eval/coverage_week{WEEK}.csv`
- `data/eval/calibration_week{WEEK}.csv`
- `data/eval/qc_summary_week{WEEK}.json` and `.txt`

> Created: 2025-09-16 19:38 


In [1]:

# === Parameters ===============================================================
SEASON = 2025
WEEK   = 2

# Files
PROPS_CSV   = f"props_with_model_week{WEEK}.csv"
ACTUALS_CSV = f"week{WEEK}.csv"
STATS_FILE  = "weekly_player_stats_2025.parquet"  # for optional backfill

# Toggle: backfill O/U actuals from weekly stats if actuals lacks them
USE_STATS_BACKFILL = True

# -----------------------------------------------------------------------------
import os, math, json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

EVAL_DIR = Path("data/eval"); EVAL_DIR.mkdir(parents=True, exist_ok=True)

print("params:", dict(SEASON=SEASON, WEEK=WEEK))
print("files:", PROPS_CSV, ACTUALS_CSV)


params: {'SEASON': 2025, 'WEEK': 2}
files: props_with_model_week2.csv week2.csv


In [2]:

# === Load ====================================================================
dfp = pd.read_csv(PROPS_CSV)
dfa = pd.read_csv(ACTUALS_CSV)
print("[props] shape:", dfp.shape, "  [actuals] shape:", dfa.shape)
dfp.head(2), dfa.head(2)


[props] shape: (2673, 24)   [actuals] shape: (160, 7)


(                            game_id         commence_time       home_team  \
 0  7d9a04f411031528d9c1d2df7b9a0453  2025-09-15T23:01:00Z  Houston Texans   
 1  7d9a04f411031528d9c1d2df7b9a0453  2025-09-15T23:01:00Z  Houston Texans   
 
               away_team                                   game   bookmaker  \
 0  Tampa Bay Buccaneers  Tampa Bay Buccaneers @ Houston Texans  draftkings   
 1  Tampa Bay Buccaneers  Tampa Bay Buccaneers @ Houston Texans  draftkings   
 
   bookmaker_title         market market_std        player  ...   player_key  \
 0      DraftKings  player_1st_td   first_td  Bucky Irving  ...  buckyirving   
 1      DraftKings  player_1st_td   first_td    Nick Chubb  ...    nickchubb   
 
    point_key  mu sigma lam  mkt_prob  model_prob  edge_bps  season  week  
 0        NaN NaN   NaN NaN  0.153846         NaN       NaN    2025     2  
 1        NaN NaN   NaN NaN  0.142857         NaN       NaN    2025     2  
 
 [2 rows x 24 columns],
   player_key name  market_st

In [5]:
import pandas as pd, unicodedata, re

# Use your normalizer (same one used in props)
try:
    from scripts.common_markets import std_name
except Exception:
    def std_name(s):
        s = unicodedata.normalize("NFKD", str(s)).encode("ascii","ignore").decode("ascii")
        return re.sub(r"[^a-z]","", str(s).lower())

# 0) Ensure join keys exist on stats
if "name_std" not in stats.columns:
    name_col_s = "player_display_name" if "player_display_name" in stats.columns else "player_name"
    stats = stats.copy()
    stats["name_std"] = stats[name_col_s].map(std_name)
if "team_std" not in stats.columns and "team" in stats.columns:
    stats["team_std"] = stats["team"].astype(str).str.upper()
if "position" not in stats.columns:
    stats["position"] = "UNK"

# 1) Map your normalized markets → stats columns
OU_MAP = {
    "pass_yds":          "passing_yards",
    "rush_yds":          "rushing_yards",
    "recv_yds":          "receiving_yards",
    "receptions":        "receptions",
    "pass_attempts":     "attempts",
    "pass_completions":  "completions",
    "rush_attempts":     "carries",
    "pass_tds":          "passing_tds",
    # note: nflverse uses 'passing_interceptions'
    "pass_ints":         "passing_interceptions",
    # only include these if your stats file actually has them:
    # "reception_longest": "reception_longest",
    # "rush_longest":      "rush_longest",
}

# keep only columns that exist in stats
have = {mkt: col for mkt, col in OU_MAP.items() if col in stats.columns}
inv = {col: mkt for mkt, col in have.items()}
value_cols = list(inv.keys())

# 2) Melt wide → long: one value column for *all* markets
s_long = (
    stats[["name_std", "team_std", "position"] + value_cols]
      .melt(id_vars=["name_std","team_std","position"],
            value_vars=value_cols,
            var_name="stat_col",
            value_name="actual_value")
      .assign(market_std=lambda d: d["stat_col"].map(inv))
      .drop(columns=["stat_col"])
      .dropna(subset=["market_std"])      # safety: only mapped markets
)

# Optional: if you only want (name_std, market_std, actual_value)
s_long_min = s_long[["name_std", "market_std", "actual_value"]]

print(s_long.head(10))
print(s_long_min.head(10))


NameError: name 'stats' is not defined

In [4]:

# === Merge with robust fallbacks =============================================
keys = []
if all(k in dfp.columns for k in ["player_key","market_std","point_key"]) and \
   all(k in dfa.columns for k in ["player_key","market_std","point_key"]):
    keys = ["player_key","market_std","point_key"]
elif all(k in dfp.columns for k in ["player_key","market_std"]) and \
     all(k in dfa.columns for k in ["player_key","market_std"]):
    keys = ["player_key","market_std"]
else:
    keys = ["name_std","market_std"]
    if "point_key" in dfp.columns and "point_key" in dfa.columns:
        keys.append("point_key")

print("merge keys:", keys)
merged = dfp.merge(dfa, how="left", on=[k for k in keys if k in dfp.columns and k in dfa.columns], suffixes=("", "_act"))
print("[merge] rows:", len(merged))
merged.head(3)


merge keys: ['player_key', 'market_std']
[merge] rows: 2829


,game_id,commence_time,home_team,away_team,game,bookmaker,bookmaker_title,market,market_std,player,...,mkt_prob,model_prob,edge_bps,season,week,name_act,side,point_act,actual_value,result
0,7d9a04f411031528d9c1d2df7b9a0453,2025-09-15T23:01:00Z,Houston Texans,Tampa Bay Buccaneers,Tampa Bay Buccaneers @ Houston Texans,draftkings,DraftKings,player_1st_td,first_td,Bucky Irving,...,0.153846,NaN,NaN,2025,2,NaN,NaN,NaN,NaN,NaN
1,7d9a04f411031528d9c1d2df7b9a0453,2025-09-15T23:01:00Z,Houston Texans,Tampa Bay Buccaneers,Tampa Bay Buccaneers @ Houston Texans,draftkings,DraftKings,player_1st_td,first_td,Nick Chubb,...,0.142857,NaN,NaN,2025,2,NaN,NaN,NaN,NaN,NaN
2,7d9a04f411031528d9c1d2df7b9a0453,2025-09-15T23:01:00Z,Houston Texans,Tampa Bay Buccaneers,Tampa Bay Buccaneers @ Houston Texans,draftkings,DraftKings,player_1st_td,first_td,Nico Collins,...,0.125000,NaN,NaN,2025,2,NaN,NaN,NaN,NaN,NaN


In [ ]:

# === Canonicalize line & side ================================================
def first_existing_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

line_src = first_existing_col(merged, ["line","point","point_key","book_line","offer_line","line_disp"])
if line_src is None:
    merged["line"] = np.nan
else:
    merged["line"] = merged[line_src]
    if merged["line"].dtype == object:
        merged["line"] = pd.to_numeric(
            merged["line"].astype(str).str.extract(r"(-?\d+(?:\.\d+)?)", expand=False),
            errors="coerce"
        )

if "side" not in merged.columns:
    guess = first_existing_col(merged, ["side","bet","selection","wager","pick"])
    merged["side"] = merged[guess] if guess else np.nan
merged["side"] = merged["side"].astype(str).str.lower().str.extract(r"(over|under|yes|no)", expand=False)

print("[canon] done. has line:", merged["line"].notna().mean(), " has side:", merged["side"].notna().mean())


In [ ]:
name_col = next((c for c in ["player_display_name","player"] if c in stats.columns), None)
if not name_col:
    raise KeyError("stats is missing a player name column")
stats = stats.copy()
stats["name_std"] = stats[name_col].map(std_name)

In [ ]:
stats['team']

In [ ]:
stats

In [ ]:
stats.columns.tolist()


In [7]:
import numpy as np

# 1) Join stats → merged on (name_std, market_std)
cols = ["name_std", "market_std", "actual_value"]
if "team_std" in s_long.columns:
    cols.append("team_std")
smin = s_long[cols].copy()

merged = merged.merge(
    smin,
    on=["name_std", "market_std"],
    how="left",
    suffixes=("", "_stats")  # stats value becomes 'actual_value_stats'
)

# 2) (Optional but recommended) Drop mismatched teams to avoid same-name collisions
if "team_std" in merged.columns:
    home = merged["home_team"].astype(str).str.upper()
    away = merged["away_team"].astype(str).str.upper()
    bad = merged["team_std"].notna() & ~merged["team_std"].isin(home.combine(away, lambda h, a: {h, a}))
    # vectorized version:
    bad = merged["team_std"].notna() & ~(
        merged["team_std"].eq(home) | merged["team_std"].eq(away)
    )
    merged.loc[bad, "actual_value_stats"] = np.nan

# 3) Fill only O/U rows that are missing actuals
OU_MARKETS = {
    "pass_yds","rush_yds","recv_yds","receptions",
    "pass_attempts","pass_completions","rush_attempts",
    "pass_tds","pass_ints","reception_longest","rush_longest"
}
need = merged["market_std"].isin(OU_MARKETS) & merged["actual_value"].isna()
merged.loc[need, "actual_value"] = merged.loc[need, "actual_value"].fillna(
    merged.loc[need, "actual_value_stats"]
)

# 4) Cleanup
if "actual_value_stats" in merged.columns:
    merged.drop(columns=["actual_value_stats"], inplace=True)


NameError: name 's_long' is not defined

In [ ]:
merged

In [ ]:
OU_MARKETS = {
    "pass_yds","rush_yds","recv_yds","receptions",
    "pass_attempts","pass_completions","rush_attempts",
    "pass_tds","pass_ints","reception_longest","rush_longest"
}
ou = merged["market_std"].isin(OU_MARKETS)

print("[O/U rows]", ou.sum())
print("[O/U with actual]", merged.loc[ou, "actual_value"].notna().sum())

# per-market audit
audit = (merged.loc[ou, ["market_std","line","actual_value"]]
         .assign(has_line=lambda d: d["line"].notna(),
                 has_actual=lambda d: d["actual_value"].notna())
         .groupby("market_std")
         .agg(rows=("market_std","size"),
              with_line=("has_line","sum"),
              with_actual=("has_actual","sum"))
         .assign(still_missing=lambda d: d["with_line"]-d["with_actual"])
         .sort_index())
print(audit)


In [ ]:
# After you load stats and filter to WEEK/SEASON:
import re, unicodedata, numpy as np, pandas as pd

def std_name(s):
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii","ignore").decode("ascii")
    return re.sub(r"[^a-z]", "", str(s).lower())

# 1) Ensure name_std (and a team if available) on stats
name_col = next((c for c in ["player_name","player_display_name","player"] if c in stats.columns), None)
if not name_col:
    raise KeyError("stats is missing a player name column")
stats = stats.copy()
stats["name_std"] = stats[name_col].map(std_name)
team_stats = next((c for c in ["recent_team","team","posteam"] if c in stats.columns), None)
if team_stats: stats["team_std"] = stats[team_stats].astype(str).str.upper()

# 2) Keep only the stat columns we can map
BACKFILL_MAP = {
    "pass_yds":         "passing_yards",
    "rush_yds":         "rushing_yards",
    "recv_yds":         "receiving_yards",
    "receptions":       "receptions",
    "pass_attempts":    "attempts",
    "pass_completions": "completions",
    "rush_attempts":    "carries",
    "pass_tds":         "passing_tds",
    "pass_ints":        "interceptions",
    "reception_longest":"reception_longest",
    "rush_longest":     "rush_longest",
}
have_map = {mkt: col for mkt, col in BACKFILL_MAP.items() if col in stats.columns}
use_cols = ["name_std"] + (["team_std"] if "team_std" in stats.columns else []) + sorted(have_map.values())
use = stats.loc[:, use_cols].drop_duplicates()

# 3) Ensure merged has compatible join cols
if "name_std" not in merged.columns:
    name_col_m = next((c for c in ["player","player_name","player_display_name","name"] if c in merged.columns), None)
    if not name_col_m: raise KeyError("merged lacks a name column to build name_std")
    merged["name_std"] = merged[name_col_m].map(std_name)
team_merged = next((c for c in ["team","recent_team","posteam"] if c in merged.columns), None)
if team_merged:
    merged["team_std"] = merged[team_merged].astype(str).str.upper()

# 4) For each market, fill actual_value by name (and team if both sides have it)
if "actual_value" not in merged.columns:
    merged["actual_value"] = np.nan

fills = {}
for mkt, stat_col in have_map.items():
    mask = (merged["market_std"].eq(mkt)) & merged["line"].notna()
    if not mask.any(): continue
    left = merged.loc[mask, ["name_std"] + (["team_std"] if "team_std" in merged.columns and "team_std" in use.columns else [])]
    right = use[["name_std"] + (["team_std"] if "team_std" in merged.columns and "team_std" in use.columns else []) + [stat_col]]
    sub = left.merge(right, on=["name_std"] + (["team_std"] if "team_std" in left.columns and "team_std" in right.columns else []), how="left")
    before = merged.loc[mask, "actual_value"].isna().sum()
    merged.loc[mask, "actual_value"] = merged.loc[mask, "actual_value"].fillna(sub[stat_col].values)
    fills[mkt] = before - merged.loc[mask, "actual_value"].isna().sum()

print("[backfill] filled:", ", ".join(f"{k}:{v}" for k,v in fills.items() if v>0) or "0")


In [ ]:

# === Compute gradeable outcomes ==============================================
model_prob_col = "model_prob" if "model_prob" in merged.columns else None

merged["pred_prob"]  = np.nan
merged["actual_bin"] = np.nan

# Binary markets
mask_bin = merged["market_std"].isin(["anytime_td","first_td","last_td"]) & (model_prob_col is not None)
if mask_bin.any():
    merged.loc[mask_bin, "pred_prob"]  = merged.loc[mask_bin, model_prob_col]
    merged.loc[mask_bin, "actual_bin"] = merged.loc[mask_bin, "actual_value"].fillna(0).clip(0,1).astype(float)

# O/U markets
OU_MARKETS = {
    "pass_yds","rush_yds","recv_yds","receptions",
    "pass_completions","pass_attempts","rush_attempts",
    "pass_tds","pass_ints"
}
mask_ou = (
    merged["market_std"].isin(OU_MARKETS) &
    merged["line"].notna() &
    merged["actual_value"].notna() &
    merged["side"].isin(["over","under"])
)

side_over  = mask_ou & merged["side"].eq("over")
side_under = mask_ou & merged["side"].eq("under")

merged.loc[side_over,  "actual_bin"] = (merged.loc[side_over,  "actual_value"] >= merged.loc[side_over,  "line"]).astype(int)
merged.loc[side_under, "actual_bin"] = (merged.loc[side_under, "actual_value"] <= merged.loc[side_under,  "line"]).astype(int)

if model_prob_col:
    merged.loc[mask_ou & merged[model_prob_col].notna(), "pred_prob"] = merged.loc[mask_ou, model_prob_col]

gradeable = merged["pred_prob"].notna() & merged["actual_bin"].notna()
graded = merged.loc[gradeable].copy()

print("[gradeable] rows:", len(graded))
print("Markets graded:\n", graded["market_std"].value_counts())
graded.head(3)


In [8]:
import pandas as pd, unicodedata, re

def std_name(s):
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii", "ignore").decode("ascii")
    return re.sub(r"[^a-z]", "", s.lower())

def ensure_player_key(df: pd.DataFrame, name_cols=("player", "player_name", "name"),
                      team_cols=("recent_team", "team", "recent_team_abbr", "posteam")) -> pd.DataFrame:
    df = df.copy()
    if "player_key" in df.columns:
        print("[ensure_player_key] already present")
        return df

    # prefer stable IDs if available
    for c in ("player_id", "gsis_id", "nfl_id", "pfr_id", "pfr_player_id"):
        if c in df.columns:
            df["player_key"] = df[c].astype(str)
            print(f"[ensure_player_key] built from {c}")
            return df

    name_col = next((c for c in name_cols if c in df.columns), None)
    team_col = next((c for c in team_cols if c in df.columns), None)

    if not name_col:
        raise KeyError("[ensure_player_key] No name column found (looked for: "
                       + ", ".join(name_cols) + ")")

    df["name_std"] = df[name_col].map(std_name)
    if team_col:
        df["player_key"] = df["name_std"] + "_" + df[team_col].astype(str).str.upper()
        src = f"{name_col}+{team_col}"
    else:
        df["player_key"] = df["name_std"]
        src = name_col

    # sanity checks
    nn = df["player_key"].notna().sum()
    uniq = df["player_key"].nunique()
    print(f"[ensure_player_key] built from {src} → non-null: {nn:,}  unique: {uniq:,}")

    # show a few examples
    print(df[[name_col] + ([team_col] if team_col else []) + ["player_key"]].head(8))
    return df

# 1) Force the backfill on *your* stats frame
stats = ensure_player_key(stats)

# 2) Optional: confirm it really exists
assert "player_key" in stats.columns and stats["player_key"].notna().any(), \
    "[assert] player_key was not created or is all NA"

# 3) If you need a rename subset:
rename_map = {
    # examples — adjust to your script
    "rushing_yards": "rush_yds",
    "receiving_yards": "rec_yds",
    "passing_yards": "pass_yds",
    "rushing_tds": "rush_tds",
    "receiving_tds": "rec_tds",
    "passing_tds": "pass_tds",
    "interceptions": "pass_ints",
}
have = [c for c in rename_map if c in stats.columns]
use = stats.loc[:, ["player_key"] + (["name_std"] if "name_std" in stats.columns else []) + have] \
           .rename(columns=rename_map, errors="ignore")

print("[use] cols →", list(use.columns))

# 4) Safe merge (replace `merged` with your props/dfp frame)
#    Prefer player_key; fallback to name_std if needed.
if "player_key" in merged.columns:
    merged = merged.merge(use, on="player_key", how="left", validate="m:1")
elif "name_std" in merged.columns and "name_std" in use.columns:
    merged = merged.merge(use.drop(columns=["player_key"]), on="name_std", how="left", validate="m:1")
else:
    raise KeyError("Neither player_key nor name_std present on both sides for merge")

print("[merge] rows:", len(merged), "  joined cols:", [c for c in use.columns if c != "player_key"])


NameError: name 'stats' is not defined

In [ ]:

# === Scoring =================================================================
def brier_score(p: np.ndarray, y: np.ndarray) -> float:
    p = np.asarray(p, dtype=float); y = np.asarray(y, dtype=float)
    return float(np.mean((p - y)**2))

def log_loss(p: np.ndarray, y: np.ndarray, eps: float = 1e-12) -> float:
    p = np.clip(np.asarray(p, dtype=float), eps, 1-eps); y = np.asarray(y, dtype=float)
    return float(-np.mean(y*np.log(p) + (1-y)*np.log(1-p)))

def hit_rate(p: np.ndarray, y: np.ndarray, threshold: float = 0.5) -> float:
    return float(np.mean(((p >= threshold).astype(int) == y.astype(int))))

if len(graded) == 0:
    raise SystemExit("No gradeable rows. Ensure model_prob, side, line, actual_value are present.")

overall = {
    "rows": len(graded),
    "hit_rate@0.5": hit_rate(graded["pred_prob"], graded["actual_bin"]),
    "brier": brier_score(graded["pred_prob"], graded["actual_bin"]),
    "logloss": log_loss(graded["pred_prob"], graded["actual_bin"]),
}
overall


In [ ]:

# === Per-market & coverage ====================================================
def summarize_by_market(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for mkt, g in df.groupby("market_std"):
        rows.append({
            "market_std": mkt,
            "rows": len(g),
            "hit_rate@0.5": float((g["pred_prob"]>=0.5).astype(int).eq(g["actual_bin"]).mean()),
            "brier": float(((g["pred_prob"]-g["actual_bin"])**2).mean()),
            "logloss": float(-np.mean(g["actual_bin"]*np.log(np.clip(g["pred_prob"],1e-12,1-1e-12))+
                                      (1-g["actual_bin"])*np.log(np.clip(1-g["pred_prob"],1e-12,1-1e-12)))),
        })
    return pd.DataFrame(rows).sort_values(["brier","logloss","rows"], ascending=[True,True,False])

by_market = summarize_by_market(graded)

cov = (
    merged.assign(is_gradeable=merged.index.isin(graded.index))
          .groupby("market_std")
          .agg(total_rows=("market_std","size"),
               gradeable_rows=("is_gradeable","sum"))
          .assign(coverage_pct=lambda d: d["gradeable_rows"]/d["total_rows"])
          .reset_index()
)

by_market.head(10), cov.sort_values("coverage_pct", ascending=False).head(15)


In [ ]:

# === Calibration ==============================================================
def calibration_bins(df: pd.DataFrame, bins=10) -> pd.DataFrame:
    cut = pd.cut(df["pred_prob"], bins=bins, include_lowest=True)
    return df.groupby(cut).agg(n=("actual_bin","size"),
                               avg_pred=("pred_prob","mean"),
                               emp_rate=("actual_bin","mean")).reset_index()

calib = calibration_bins(graded, bins=10)
plt.figure()
plt.plot(calib["avg_pred"], calib["emp_rate"], marker="o")
plt.plot([0,1],[0,1], linestyle="--")
plt.xlabel("Predicted probability (bin avg)"); plt.ylabel("Empirical win rate")
plt.title(f"Calibration — Week {WEEK}"); plt.grid(True); plt.show()

calib.head(10)


In [ ]:

# === Save outputs & quick summary ============================================
graded.to_csv(EVAL_DIR/f"grades_week{WEEK}.csv", index=False)
by_market.to_csv(EVAL_DIR/f"grades_week{WEEK}_by_market.csv", index=False)
cov.to_csv(EVAL_DIR/f"coverage_week{WEEK}.csv", index=False)
calib.to_csv(EVAL_DIR/f"calibration_week{WEEK}.csv", index=False)

summary = (
    f"Week {WEEK} — graded {overall['rows']} props. "
    f"Hit-rate@0.5={overall['hit_rate@0.5']:.1%}, "
    f"Brier={overall['brier']:.3f}, LogLoss={overall['logloss']:.3f}. "
)
best = by_market.sort_values("brier").head(3).to_dict("records")
worst = by_market.sort_values("brier").tail(3).to_dict("records")
summary += "Best: " + ", ".join(f"{r['market_std']} (Brier {r['brier']:.3f}, n={r['rows']})" for r in best) + ". "
summary += "Needs work: " + ", ".join(f"{r['market_std']} (Brier {r['brier']:.3f}, n={r['rows']})" for r in worst) + "."

print(summary)

# shareables
(EVAL_DIR/f"qc_summary_week{WEEK}.json").write_text(json.dumps({"week":WEEK, "overall":overall,
                                                               "best":best, "worst":worst}, indent=2))
(EVAL_DIR/f"qc_summary_week{WEEK}.txt").write_text(summary)

print(f"[write] {EVAL_DIR}/grades_week{WEEK}.csv")
print(f"[write] {EVAL_DIR}/grades_week{WEEK}_by_market.csv")
print(f"[write] {EVAL_DIR}/coverage_week{WEEK}.csv")
print(f"[write] {EVAL_DIR}/calibration_week{WEEK}.csv")
print(f"[write] {EVAL_DIR}/qc_summary_week{WEEK}.json / .txt")
